In [ ]:
import cv2  # OpenCV para procesamiento de video
import mediapipe as mp  # Mediapipe para detección de manos
import screen_brightness_control as sbc  # Control de brillo
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume  # Control de audio
from comtypes import CLSCTX_ALL  # Requisito para Pycaw
import tkinter as tk  # Interfaz gráfica
from tkinter import BooleanVar
import threading  # Hilos para ejecutar detección en paralelo
import time  # Para agregar temporización
import numpy as np  # Para manejar los puntos de la mano

# Inicialización de control de volumen
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
endpoint_volume = interface.QueryInterface(IAudioEndpointVolume)

# Función para setear el volumen principal del sistema
def set_volume(volume_level):
    """
    Ajusta el volumen del sistema en un rango de 0 a 100.
    :param volume_level: Nivel de volumen (int entre 0 y 100).
    """
    volume_level = max(0, min(100, volume_level))  # Asegura que esté entre 0 y 100
    endpoint_volume.SetMasterVolumeLevelScalar(volume_level / 100.0, None)

# Función para iniciar la detección de manos
def start_detection():
    global running
    running = True
    threading.Thread(target=hand_detection).start()

# Función para detener la detección de manos
def stop_detection():
    global running
    running = False

# Función principal de detección de manos
def hand_detection():
    global running, volume_label, brightness_label, show_video_var
    # Modelo de detección de manos Mediapipe
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)
    cap = cv2.VideoCapture(0)

    control_state = 'volume'  # Estado inicial del control (volumen)
    last_switch_time = time.time()  # Tiempo del último cambio de control
    switch_delay = 2  # Espera de 2 segundos antes de permitir otro cambio
    alignment_threshold = 0.1  # Diferencia mínima entre el pulgar y el meñique para alternar

    while running and cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Imagen captada
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Conversión a RGB
        results = hands.process(rgb_frame)  # Detección de la mano

        if results.multi_hand_landmarks:
            for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                mp_drawing = mp.solutions.drawing_utils
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                label = handedness.classification[0].label  # Distinción entre derecha e izquierda

                if label == 'Right':  # Solo usamos la mano derecha
                    # Detectar si la mano está de perfil usando la alineación del pulgar con el meñique
                    thumb_x = hand_landmarks.landmark[4].x
                    pinky_x = hand_landmarks.landmark[20].x

                    # Si la diferencia en la posición horizontal del pulgar y meñique es pequeña
                    if abs(thumb_x - pinky_x) < alignment_threshold:
                        current_time = time.time()
                        if current_time - last_switch_time >= switch_delay:
                            # Solo cambiar de control si ha pasado el tiempo de espera
                            if control_state == 'volume':
                                control_state = 'brightness'
                            else:
                                control_state = 'volume'
                            last_switch_time = current_time  # Actualizar el tiempo del último cambio

                    # Ajustar volumen o brillo según la cantidad de dedos levantados
                    vol = 0
                    bright = 0
                    finger_count = 0  # Contador de dedos levantados

                    # Se comprueba si la punta de cada dedo está por encima de la segunda articulación
                    for i in range(1, 5):  # Revisamos todos los dedos excepto el pulgar
                        tip_y = hand_landmarks.landmark[4 + 4 * i].y
                        pip_y = hand_landmarks.landmark[3 + 4 * i].y
                        if tip_y < pip_y:  # Si la punta del dedo está por encima de la segunda articulación
                            finger_count += 1

                    # Detectar el pulgar de manera más robusta, sin interferir con la alineación
                    thumb_tip_y = hand_landmarks.landmark[4].y
                    thumb_ip_y = hand_landmarks.landmark[3].y
                    # Solo contamos el pulgar si está levantado (punta por encima de la segunda articulación)
                    if thumb_tip_y < thumb_ip_y:  
                        finger_count += 1

                    # Si no se detectan dedos levantados, la cantidad es 0
                    if finger_count == 0:
                        vol = 0
                        bright = 0
                    else:
                        # Mapeo de dedos levantados a 0-100%
                        percentage = int(100 * finger_count / 5)  # 0-100% basado en el número de dedos levantados
                        if control_state == 'volume':
                            vol = percentage  # Volumen en porcentaje
                            set_volume(vol)
                            volume_label.config(text=f"Volumen: {vol}%")
                        elif control_state == 'brightness':
                            bright = percentage  # Brillo en porcentaje
                            sbc.set_brightness(bright)
                            brightness_label.config(text=f"Brillo: {bright}%")

                    # Mostrar el número de dedos en la pantalla
                    cv2.putText(frame, f'Dedos detectados: {finger_count}', 
                                (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

                    # Mostrar texto sobre el marco
                    cv2.putText(frame, f'Control: {control_state.capitalize()} ({vol if control_state == "volume" else bright}% ajuste)', 
                                (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # Mostrar el video si está habilitado
        if show_video_var.get():
            cv2.imshow('Hand Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Salir con la tecla 'q'
            break

    cap.release()
    cv2.destroyAllWindows()
    hands.close()

# Configuración de la interfaz gráfica
app = tk.Tk()
app.title("Control de Volumen y Brillo")
app.geometry("400x300")

# Etiquetas y botones de control
volume_label = tk.Label(app, text="Volumen: 50%", font=("Helvetica", 12))
volume_label.pack(pady=10)

brightness_label = tk.Label(app, text="Brillo: 50%", font=("Helvetica", 12))
brightness_label.pack(pady=10)

start_button = tk.Button(app, text="Iniciar Detección", command=start_detection, bg="green", fg="white")
start_button.pack(pady=10)

stop_button = tk.Button(app, text="Detener Detección", command=stop_detection, bg="red", fg="white")
stop_button.pack(pady=10)

# Checkbox para mostrar/ocultar video
show_video_var = BooleanVar(value=True)
show_video_checkbox = tk.Checkbutton(app, text="Mostrar Video", variable=show_video_var)
show_video_checkbox.pack(pady=10)

# Variable para controlar el hilo de detección
running = False

# Ejecuta la interfaz
app.mainloop()
